In [18]:
import json
from pprint import pprint
from analysis import gender

ImportError: No module named analysis

In [2]:
def process_coref_chain(coref_id, coref_chain):
    # Get the unique representative element
    rep_elem = next(coref for coref in coref_chain if coref['isRepresentativeMention'])
    to_print = [(elem['text'], elem['type'], elem['number'], elem['gender'], elem['sentNum']) for elem in coref_chain
                if elem['animacy'] == 'ANIMATE']
    if len(to_print):
        print len(coref_chain), to_print

In [3]:
def process_sentences(sentences):
    """
    Process the sentences object returned by CoreNLP's annotation.
    """
    people_mentioned = set()
    curr_mention = ''
    for (i, sentence) in enumerate(sentences):
        assert i == sentence['index']
        tokens = sentence['tokens']
        for token in tokens:
            if token['ner'] == 'PERSON':
                if len(curr_mention) > 0:
                    curr_mention += ' '
                curr_mention += token['originalText']
            else:
                if len(curr_mention) > 0:
                    sp_cm = tuple(curr_mention.split())
                    intersected = False
                    to_add = []
                    to_remove = []
                    for pm in people_mentioned:
                        if len(set.intersection(set(pm), set(sp_cm))) > 0:
                            intersected = True
                            if len(sp_cm) > len(pm):
                                to_add.append(sp_cm)
                                to_remove.append(pm)
                    if not intersected:
                        people_mentioned.add(sp_cm)
                    else:
                        for elem in to_remove:
                            people_mentioned.remove(elem)
                        for elem in to_add:
                            people_mentioned.add(elem)
                    curr_mention = ''
    
    return people_mentioned

In [4]:
with open('data/annotated_old/techcrunch_annotated_2016.json', 'r') as tc_f:
    tc_2016_data = json.load(tc_f)

In [6]:
def get_mentions_counts(data):
    counts = {'MALE': 0, 'FEMALE': 0}
    for link, data in data.iteritems():
        try:
            sentences = data['corenlp']['sentences']
            corefs = data['corenlp']['corefs']
            people_mentioned = process_sentences(sentences)
            pm_to_gender = {pm: None for pm in people_mentioned}
            for c_id, coref_chain in corefs.iteritems():
                rep_elem = next(coref for coref in coref_chain if coref['isRepresentativeMention'])
                elem_text = set(rep_elem['text'].split())
                for pm in pm_to_gender:
                    if len(set.intersection(elem_text, set(pm))) > 0:
                        if rep_elem['gender'] in ['MALE', 'FEMALE']:
                            pm_to_gender[pm] = rep_elem['gender']
            for gender in ['MALE', 'FEMALE']:
                counts[gender] += len([sp for sp in pm_to_gender.values() if sp == gender])
        except:
            continue
    return counts

In [7]:
for year in range(2013, 2017):
    with open('data/annotated_old/techcrunch_annotated_{}.json'.format(year), 'r') as tc_f:
        data = json.load(tc_f)
        print year, get_mentions_counts(data)

2013 {'MALE': 27925, 'FEMALE': 3193}
2014 {'MALE': 22635, 'FEMALE': 2913}
2015 {'MALE': 21244, 'FEMALE': 3024}
2016 {'MALE': 12415, 'FEMALE': 1875}


In [15]:
MIN_NUM = 623 + 6
MAX_NUM = MIN_NUM + 1
curr_idx = 0
for link, data in tc_2016_data.iteritems():
    print data['author']
    curr_idx += 1
    if curr_idx < MIN_NUM:
        continue
    if curr_idx >= MAX_NUM:
        break    
    print link
    corefs = data['corenlp']['corefs']
    sentences = data['corenlp']['sentences']
    # print sentences[0].keys()
    # pprint(sentences[5]['tokens'])
    print process_sentences(sentences)
    for c_id, coref in corefs.iteritems():
        process_coref_chain(c_id, coref)

Mark Kaganovich
Natasha Lomas
Ingrid Lunden
Matthew Lynley
Jonathan Shieber
Ben Narasin
Robert Dale
Matthew Panzarino
Romain Dillet
Romain Dillet
Simon Khalaf
Kristen Hall-Geisler
Jonathan Shieber
Lora Kolodny
Kate Conger
Romain Dillet
Connie Loizos
Katie Roof
Catherine Shu
Lucas Matney
Sarah Buhr
Jon Russell
Jon Russell
Ron Miller
Joe Edelheit Ross
Anthony Ha
Peter Hirst
Matt Burns
Darrell Etherington
Connie Loizos
Sarah Perez
Megan Rose Dickey
Jake Bright
Ryan Angilly
Darrell Etherington
Anna Escher
Lucas Matney
Benjamin Brandall
Lora Kolodny
Charles Birnbaum
Ingrid Lunden
Catherine Shu
Sarah Buhr
Haje Jan Kamps
Sarah Buhr
Connie Loizos
Evan Baehr
Josh Constine
Natasha Lomas
Lucas Matney
Josh Constine
Lora Kolodny
Connie Loizos
Mark Lelinwalla
Stefan Etienne
Sarah Buhr
Lucas Matney
Matt Burns
Jonathan Shieber
Jordan Crook
Romain Dillet
Darrell Etherington
Jon Evans
Mike Butcher
Sarah Perez
Anthony Ha
Josh Constine
Ingrid Lunden
Jake Bright
Samantha O'Keefe
Frederic Lardinois
Sarah Bu

https://techcrunch.com/2016/06/20/is-enterprise-genomics-good-enough-yet/
